In [1]:
import numpy as np
import pandas as pd

In [2]:
# read in excel data
df = pd.read_excel('Data for Coding Test.xlsx', sheet_name='problem #1', header=None, dtype=float, skiprows=[0, 1], usecols="B:BH")
rets = df.to_numpy() # return data
rets.shape

(522, 59)

In [3]:
# weights column vector
d = 0.01
w = [(1-d)**((i-1)/2) for i in range (len(rets),0,-1)]
w = np.array(w)
w.shape

(522,)

In [4]:
# r_hatD
r_hatD = np.transpose(np.transpose(rets)*w)
r_hatD
r_hatD.shape

(522, 59)

In [5]:
# s_0hatD
s_0hatD = np.dot(np.transpose(r_hatD), r_hatD) / np.sum(w)
s_0hatD
s_0hatD.shape

(59, 59)

In [6]:
# r_hat1D
r_hat1D = np.append(np.array([[0]*len(r_hatD[0])]), r_hatD[:-1], axis=0)
r_hat1D
r_hat1D.shape

(522, 59)

In [7]:
# s_1hatD
s_1hatD = np.dot(np.transpose(r_hatD), r_hat1D) / np.sum(w)
s_1hatD
s_1hatD.shape

(59, 59)

In [8]:
# r_hat2D
r_hat2D = np.append(np.array([[0]*len(r_hatD[0]),[0]*len(r_hatD[0])]), r_hatD[:-2], axis=0)
r_hat2D
r_hat2D.shape

(522, 59)

In [9]:
# s_2hatD
s_2hatD = np.dot(np.transpose(r_hatD), r_hat2D) / np.sum(w)
s_2hatD
s_2hatD.shape

(59, 59)

In [10]:
# s_hatM
p = 21
s_hatM = s_0hatD*p + np.add(s_1hatD,np.transpose(s_1hatD))*(p-1) + np.add(s_2hatD,np.transpose(s_2hatD))*(p-2)
s_hatM
s_hatM.shape

(59, 59)

In [11]:
# read in risk contribution
risk_df = pd.read_excel('Data for Coding Test.xlsx', sheet_name='problem #1', header=None, usecols="B:BH").iloc[0]
risk_con= risk_df.to_numpy().astype(float)
risk_con.shape

(59,)

In [12]:
vol = 0.1
var = vol**2
risk_con = risk_con*var # the risk vector
risk_con.shape

(59,)

In [13]:
# Optimize using scipy
# Diagonal Matrix with weights on the diag
from scipy import optimize
def fit(x,a):
    return np.dot(np.dot(np.diagflat(x),a),x)
def err(x):
    return fit(x,s_hatM)-risk_con

In [14]:
start = np.array([0]*59) # initial guess, 59 is a magic number, which is the number of assets
res = optimize.leastsq(err,start)
res

(array([ 2.21964573e-01,  1.95469004e-01,  1.84675678e-01,  5.57576450e-02,
         4.50040067e-02,  3.81518959e-02,  4.08945449e-02,  3.50823877e-02,
         3.11133028e-02,  5.21060448e-02,  3.60352144e-02,  5.76116367e-02,
         4.39082589e-02,  5.18587138e-02,  4.81718628e-02,  6.36768531e-02,
         6.59791553e-02,  8.43456269e-02,  4.98202716e-02,  8.46132526e-02,
         8.11676332e-02,  4.72205134e-02,  4.66249368e-02, -1.45554661e-01,
         5.34535303e-02,  7.61106099e-02,  5.13700786e-02,  2.05332462e-01,
         6.04167282e-02,  7.46472946e-02,  3.72282123e-02,  4.15823191e-02,
         4.07293667e-02,  4.25973314e-02,  7.72569839e-02,  7.12440822e-02,
         4.20537804e-02,  1.38126139e-01,  2.28603727e-01,  1.35467489e-01,
         5.85751204e-02,  6.83020388e-02,  5.60155858e-02,  8.45125944e-02,
         1.33986651e-01,  7.94844045e-16,  9.35492646e-01,  5.86458248e-16,
         6.74675565e-01,  6.53039873e-01,  4.10338898e-01,  2.64827031e-16,
         6.2

In [15]:
weights = res[0] # calculated weights
weights

array([ 2.21964573e-01,  1.95469004e-01,  1.84675678e-01,  5.57576450e-02,
        4.50040067e-02,  3.81518959e-02,  4.08945449e-02,  3.50823877e-02,
        3.11133028e-02,  5.21060448e-02,  3.60352144e-02,  5.76116367e-02,
        4.39082589e-02,  5.18587138e-02,  4.81718628e-02,  6.36768531e-02,
        6.59791553e-02,  8.43456269e-02,  4.98202716e-02,  8.46132526e-02,
        8.11676332e-02,  4.72205134e-02,  4.66249368e-02, -1.45554661e-01,
        5.34535303e-02,  7.61106099e-02,  5.13700786e-02,  2.05332462e-01,
        6.04167282e-02,  7.46472946e-02,  3.72282123e-02,  4.15823191e-02,
        4.07293667e-02,  4.25973314e-02,  7.72569839e-02,  7.12440822e-02,
        4.20537804e-02,  1.38126139e-01,  2.28603727e-01,  1.35467489e-01,
        5.85751204e-02,  6.83020388e-02,  5.60155858e-02,  8.45125944e-02,
        1.33986651e-01,  7.94844045e-16,  9.35492646e-01,  5.86458248e-16,
        6.74675565e-01,  6.53039873e-01,  4.10338898e-01,  2.64827031e-16,
        6.23303016e-01,  

In [16]:
achieved = np.dot(np.dot(np.diagflat(weights),s_hatM),weights) # risk weight achieved
achieved_weight = achieved/np.sum(achieved)
achieved_weight

array([3.70370370e-02, 3.70370370e-02, 3.70370370e-02, 1.01010101e-02,
       1.01010101e-02, 1.01010101e-02, 1.01010101e-02, 1.01010101e-02,
       1.01010101e-02, 1.01010101e-02, 1.01010101e-02, 1.01010101e-02,
       1.01010101e-02, 1.01010101e-02, 1.58730159e-02, 1.58730159e-02,
       1.58730159e-02, 1.58730159e-02, 1.58730159e-02, 1.58730159e-02,
       1.58730159e-02, 1.85185185e-02, 1.85185185e-02, 1.85185185e-02,
       1.85185185e-02, 1.85185185e-02, 1.85185185e-02, 1.01010101e-02,
       1.01010101e-02, 1.01010101e-02, 1.01010101e-02, 1.01010101e-02,
       1.01010101e-02, 1.01010101e-02, 1.01010101e-02, 1.01010101e-02,
       1.01010101e-02, 1.01010101e-02, 1.58730159e-02, 1.58730159e-02,
       1.58730159e-02, 1.58730159e-02, 1.58730159e-02, 1.58730159e-02,
       1.58730159e-02, 1.29210333e-17, 3.70370370e-02, 2.29290640e-17,
       3.70370370e-02, 3.70370370e-02, 3.70370370e-02, 3.09993806e-18,
       3.70370370e-02, 3.70370370e-02, 6.33267876e-18, 7.91734949e-18,
      

In [17]:
#Output
with pd.ExcelWriter('p1_out.xlsx', engine="openpyxl") as writer:
    pd.DataFrame(s_hatM).to_excel(writer, sheet_name='Covariance_Estimate_Sheet')
    pd.DataFrame(weights).to_excel(writer, sheet_name='Weights_Sheet')
    pd.DataFrame(achieved_weight).to_excel(writer, sheet_name='Achieved_Risk_Weights_Sheet')
print("finished")

finished
